In [427]:
import os
import random
import pickle
import networkx as nx
import math

In [428]:
def load_random_graph(graphs_folder):
    graph_files = [f for f in os.listdir(graphs_folder) if f.endswith('.pkl')]
    random_graph_file = random.choice(graph_files)
    with open(os.path.join(graphs_folder, random_graph_file), 'rb') as f:
        G = pickle.load(f)
    return G, random_graph_file

# Putanja do foldera sa grafovima
graphs_folder = 'grafovi'

# Učitavanje grafa
G_global, graph_file_name = load_random_graph(graphs_folder)
# G_info = nx.info(G)
num_nodes = G_global.number_of_nodes()

# G_info, num_nodes, graph_file_name
num_nodes, graph_file_name

(87, 'graf_25.pkl')

In [429]:
def solution_quality(solution):
    return len(solution.nodes())

In [430]:
def initialize_solution(G, k):
    while True:
        potential_solution = G.copy()
        nodes_to_remove = random.sample(potential_solution.nodes(), k)
        potential_solution.remove_nodes_from(nodes_to_remove)
        if is_k_connected(potential_solution, k):
            return potential_solution

In [431]:
def local_search(G, k, current_solution):
    # Ova funkcija treba da smanji broj čvorova u rešenju, ako je to moguće
    # Proverava da li može da ukloni bilo koji čvor a da i dalje ostane k-vezan
    print("Broj cvorova na ulasku u local_search() ")
    print(current_solution.number_of_nodes())
    temp_solution = current_solution.copy()
    nodes = list(current_solution.nodes())
    random.shuffle(nodes)
    for node in nodes:
        temp_solution.remove_node(node)
        if not is_k_connected(temp_solution, k):
            temp_solution.add_node(node)
    print("Broj cvorova na izlasku iz local_search() ")
    print(temp_solution.number_of_nodes())
    return temp_solution

In [432]:
def is_k_connected(G, k):
    if G.number_of_nodes() < k:
        return False
    else:
        return nx.node_connectivity(G) >= k


In [433]:
def shaking(G, k, current_solution):
    # Dodavanje ili uklanjanje čvorova ili ivica
    modified_solution = current_solution.copy()
    if random.choice([True, False]):
        # Ukloni slučajno odabrane čvorove, ali ne manje od k
        nodes_to_remove = random.sample(modified_solution.nodes(), k)
        modified_solution.remove_nodes_from(nodes_to_remove)
    else:
        # Dodaj slučajno odabrane čvorove iz originalnog grafa
        potential_nodes = list(set(G.nodes()) - set(modified_solution.nodes()))
        nodes_to_add = random.sample(potential_nodes, math.floor(len(potential_nodes)/2))
        modified_solution.add_nodes_from(nodes_to_add)
        # Dodaj i odgovarajuće ivice
        for node in nodes_to_add:
            for neighbor in G.neighbors(node):
                if neighbor in modified_solution:
                    modified_solution.add_edge(node, neighbor)
    
    # Provera da li je i dalje k-vezan
    print(f'shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: {modified_solution.number_of_nodes()}')
    modified_solution = remove_nodes_with_less_than_k_neighbors_optimized(modified_solution, k)
    print(f'shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: {modified_solution.number_of_nodes()}')
    if is_k_connected(modified_solution, k):
        print("shaking: modifikovano resenje je prihvaljivo")
        return modified_solution
    else:
        print("shaking: modifikovano resenje nije prihvaljivo")
        return current_solution


In [434]:
def remove_nodes_with_less_than_k_neighbors_optimized(G, k):
    changed = True
    while changed:
        changed = False
        nodes_to_remove = [node for node in G.nodes if G.degree(node) < k]
        if nodes_to_remove:
            G.remove_nodes_from(nodes_to_remove)
            changed = True
    return G

In [435]:
from time import perf_counter

In [436]:
def vns_algorithm(G, k, max_iter=5):
    # Inicijalizacija
    # current_solution = initialize_solution(G, k)
    # print(current_solution.number_of_nodes())
    start_time = perf_counter()
    G = remove_nodes_with_less_than_k_neighbors(G, k)
    best_solution = G.copy()
    print(f'vns_algorithm: broj cvorova na pocetku: {G.number_of_nodes()}')
    while perf_counter() - start_time < 2*60:
        # Shaking
        shaken_solution = shaking(G, k, best_solution)
        # Local search
        new_solution = local_search(G, k, shaken_solution)
        
        # Ažuriranje trenutnog rešenja ako je novo rešenje bolje
        if solution_quality(new_solution) < solution_quality(best_solution):
            best_solution = new_solution.copy()
            print(f'vns_algorithm: nadjeno je novo najbolje resenje: {best_solution.number_of_nodes()}')
    
    return best_solution

In [437]:
final_solution = vns_algorithm(G_global, 5, 10)
print("Broj cvorova poslednjeg grafa")
print(final_solution.number_of_nodes())


vns_algorithm: broj cvorova na pocetku: 87
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 87
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 87
shaking: modifikovano resenje je prihvaljivo
Broj cvorova na ulasku u local_search() 
87
Broj cvorova na izlasku iz local_search() 
34
vns_algorithm: nadjeno je novo najbolje resenje: 34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 29
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
Broj cvorova na ulasku u local_search() 
34
Broj cvorova na izlasku iz local_search() 
34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 29
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano resenje nije prihvaljivo
Broj cvorova na ulasku u local_search() 
34
Broj cvorova na izlasku iz local_search() 
34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 29
shaking: 

In [376]:
from itertools import combinations

def brute_force_algorithm(G, k):
    all_nodes = list(G.nodes())
    n = len(all_nodes)
    best_solution = None
    best_size = float('inf')

    # Provera svih mogućih kombinacija čvorova
    for r in range(n, k-1, -1):
        for nodes in combinations(all_nodes, r):
            subgraph = G.subgraph(nodes)
            if nx.is_k_edge_connected(subgraph, k):
                if r < best_size:
                    best_solution = subgraph
                    best_size = r

    return best_solution